In [8]:
import pandas as pd
import numpy as np
import random, os
import plotly.express as px

import tqdm
from datetime import datetime

In [9]:
# 量的データの標準化
from sklearn.preprocessing import StandardScaler

# K-Prototypeクラスタリング
from kmodes.kprototypes import KPrototypes

# Gower距離
import gower

# 階層クラスタリング
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram

In [10]:
random_state = 42
os.environ['PYTHONHASHSEED'] = str(random_state)
random.seed(random_state)
np.random.seed(random_state)


In [11]:
os.getcwd()

'/Users/3ign0n/workspace/data-science/SIGNATE/SIGNATE_Career_Up_Challenge_2023/scuc23/notebooks'

In [12]:
df0 = pd.read_csv('../data/01_raw/train.csv')
df0

,id,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price
0,0,nashville,1949,bmw,excellent,6 cylinders,gas,115148,clean,manual,rwd,mid-size,convertible,orange,NaN,27587
1,1,state college,2013,toyota,fair,8 cylinders,gas,172038,clean,automatic,rwd,full-size,sedan,silver,pa,4724
2,2,wichita,1998,ford,good,6 cylinders,gas,152492,clean,automatic,fwd,full-size,SUV,silver,ks,10931
3,3,albany,2014,ford,excellent,4 cylinders,gas,104118,clean,manual,fwd,mid-size,SUV,blue,ny,16553
4,4,redding,2005,ford,excellent,6 cylinders,gas,144554,clean,manual,fwd,mid-size,sedan,red,ca,5158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,27527,williamsport,2008,ford,good,6 cylinders,gas,26660,clean,automatic,rwd,compact,truck,black,pa,32212
27528,27528,tulsa,2007,ford,excellent,8 cylinders,gas,108072,clean,automatic,rwd,full-size,pickup,black,NaN,5400
27529,27529,rochester,2019,jeep,like new,6 cylinders,gas,139908,clean,automatic,4wd,mid-size,SUV,white,ny,22227
27530,27530,rochester,2007,jeep,excellent,6 cylinders,gas,112326,clean,automatic,4wd,mid-size,sedan,white,ny,3054


In [13]:
df1 = pd.read_csv('../data/01_raw/test.csv')
df1

,id,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
0,27532,western slope,2015,chevrolet,excellent,4 cylinders,gas,92553,clean,automatic,fwd,full-size,SUV,red,NaN
1,27533,roseburg,2013,nissan,like new,4 cylinders,gas,134385,salvage,automatic,fwd,mid-size,sedan,black,or
2,27534,akron / canton,2011,volkswagen,good,4 cylinders,gas,102489,clean,automatic,fwd,full-size,sedan,black,oh
3,27535,denver,2016,jeep,excellent,6 cylinders,diesel,64310,clean,automatic,4wd,mid-size,SUV,red,co
4,27536,hickory / lenoir,1999,honda,excellent,8 cylinders,gas,180839,rebuilt,automatic,4wd,mid-size,SUV,silver,nc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27532,55064,great falls,2016,gmc,excellent,6 cylinders,gas,90902,rebuilt,automatic,fwd,full-size,truck,black,mt
27533,55065,las vegas,2012,nissan,excellent,4 cylinders,gas,27234,rebuilt,automatic,fwd,mid-size,sedan,white,nv
27534,55066,phoenix,2002,bmw,excellent,6 cylinders,gas,99761,clean,automatic,rwd,full-size,coupe,blue,az
27535,55067,colorado springs,2006,bmw,excellent,6 cylinders,gas,162279,clean,automatic,4wd,mid-size,sedan,silver,co


In [14]:
df_region_state = pd.read_csv('../data/01_raw/region-state.csv')
df_region_state

,region,state
0,nashville,tn
1,rockford,il
2,san antonio,tx
3,flagstaff / sedona,az
4,las vegas,nv
...,...,...
342,glens falls,ny
343,olympic peninsula,wa
344,southeast KS,ks
345,janesville,wi


In [15]:
def __preprocess_column_region_state(data: pd.DataFrame, region_state_data: pd.DataFrame) -> pd.DataFrame:
    tmp_df = pd.merge(data, region_state_data, on='region', how='left')
    tmp_df['state']=tmp_df['state_x'].fillna(tmp_df['state_y'])
    tmp_df = tmp_df.drop(columns=['state_x', 'state_y'])
    return tmp_df

def __preprocess_column_year(data: pd.DataFrame) -> pd.DataFrame:
    # 2999年以上の値はおかしいので、入力ミスと考え、-1000する
    data.loc[data['year']>=2999, 'year'] = data['year'] - 1000
    return data

def __preprocess_column_manufacturer(data: pd.DataFrame) -> pd.DataFrame:
    data['manufacturer'] = data['manufacturer'].str.normalize('NFKC')
    return data


def __preprocess_column_odometer(data: pd.DataFrame) -> pd.DataFrame:
    # オドメーターが負数というのはおかしいので、入力ミスと考え、正数に直す
    data.loc[data['odometer']<0, 'odometer'] = data['odometer'] * -1
    return data


def __preprocess_column_size(data: pd.DataFrame) -> pd.DataFrame:
    data['size'] = data['size'].str.replace('ー', '-')
    data['size'] = data['size'].str.replace('−', '-')    
    return data

def __preprocess_standarize(data: pd.DataFrame) -> pd.DataFrame:
    numerical = data[['year', 'odometer']]

    stdscaler = StandardScaler()
    transformed = pd.DataFrame(stdscaler.fit_transform(numerical), columns=numerical.columns)
    data['year']=transformed['year']
    data['odometer']=transformed['odometer']
    return data
    
def __preprocess_nan(data: pd.DataFrame) -> pd.DataFrame:
    # クラスタリングする上では、NaNが許されないようなので、適当に埋める
    # fuel, title_status, type, stateにNaNが見つかった
    # fuel: gasが2万数千で圧倒的に多く、次がdieselで2千数百。ほかは数十なので、gasにしておけば正しい確率が高そう
    # title_status: ざっと見た感じ、title_satatusによらず、高いものは高いし、安いものは安い様に見えた。2万2千超えでほとんどがcleanなので、それにしておけば当たっている確率が高そう
    # used carのtitleとは何か？は↓参照
    # https://www.caranddriver.com/research/a32811335/types-of-car-titles/
    # type: sedanが9千5百、SUVが7千強、truckが4千弱
    # state: regionから特定可能なので、それで埋める
    data['fuel'] = data['fuel'].fillna('gas')
    data['title_status'] = data['title_status'].fillna('clear')

    # FIXME: ここは雑なので、後で考える
    data['type'] = data['type'].fillna('sedan')
    return data

def __apply_preprocessing_rules(data: pd.DataFrame, region_state_data: pd.DataFrame) -> pd.DataFrame:
    tmp_df = __preprocess_column_region_state(data, region_state_data)
    tmp_df = __preprocess_column_year(tmp_df)
    tmp_df = __preprocess_column_manufacturer(tmp_df)
    tmp_df = __preprocess_column_odometer(tmp_df)
    tmp_df = __preprocess_column_size(tmp_df)
    tmp_df = __preprocess_standarize(tmp_df)
    return __preprocess_nan(tmp_df)


# id、priceはクラスタリングには使わない
df_train = __apply_preprocessing_rules(df0, df_region_state).drop('id', axis=1).drop('price', axis=1)
df_test = __apply_preprocessing_rules(df0, df_region_state).drop('id', axis=1)

In [16]:
df_train

,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
0,nashville,-5.797528,bmw,excellent,6 cylinders,gas,-0.013666,clean,manual,rwd,mid-size,convertible,orange,tn
1,state college,0.524784,toyota,fair,8 cylinders,gas,0.868377,clean,automatic,rwd,full-size,sedan,silver,pa
2,wichita,-0.957008,ford,good,6 cylinders,gas,0.565328,clean,automatic,fwd,full-size,SUV,silver,ks
3,albany,0.623570,ford,excellent,4 cylinders,gas,-0.184679,clean,manual,fwd,mid-size,SUV,blue,ny
4,redding,-0.265505,ford,excellent,6 cylinders,gas,0.442255,clean,manual,fwd,mid-size,sedan,red,ca
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,williamsport,0.030854,ford,good,6 cylinders,gas,-1.385616,clean,automatic,rwd,compact,truck,black,pa
27528,tulsa,-0.067933,ford,excellent,8 cylinders,gas,-0.123375,clean,automatic,rwd,full-size,pickup,black,ok
27529,rochester,1.117501,jeep,like new,6 cylinders,gas,0.370222,clean,automatic,4wd,mid-size,SUV,white,ny
27530,rochester,-0.067933,jeep,excellent,6 cylinders,gas,-0.057420,clean,automatic,4wd,mid-size,sedan,white,ny


In [17]:
df_train.isnull().any()

region          False
year            False
manufacturer    False
condition       False
cylinders       False
fuel            False
odometer        False
title_status    False
transmission    False
drive           False
size            False
type            False
paint_color     False
state           False
dtype: bool

In [16]:
df_train.isna().any()

region          False
year            False
manufacturer    False
condition       False
cylinders       False
fuel            False
odometer        False
title_status    False
transmission    False
drive           False
size            False
type            False
paint_color     False
state           False
dtype: bool

In [18]:
df_test

,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,price,state
0,nashville,-5.797528,bmw,excellent,6 cylinders,gas,-0.013666,clean,manual,rwd,mid-size,convertible,orange,27587,tn
1,state college,0.524784,toyota,fair,8 cylinders,gas,0.868377,clean,automatic,rwd,full-size,sedan,silver,4724,pa
2,wichita,-0.957008,ford,good,6 cylinders,gas,0.565328,clean,automatic,fwd,full-size,SUV,silver,10931,ks
3,albany,0.623570,ford,excellent,4 cylinders,gas,-0.184679,clean,manual,fwd,mid-size,SUV,blue,16553,ny
4,redding,-0.265505,ford,excellent,6 cylinders,gas,0.442255,clean,manual,fwd,mid-size,sedan,red,5158,ca
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,williamsport,0.030854,ford,good,6 cylinders,gas,-1.385616,clean,automatic,rwd,compact,truck,black,32212,pa
27528,tulsa,-0.067933,ford,excellent,8 cylinders,gas,-0.123375,clean,automatic,rwd,full-size,pickup,black,5400,ok
27529,rochester,1.117501,jeep,like new,6 cylinders,gas,0.370222,clean,automatic,4wd,mid-size,SUV,white,22227,ny
27530,rochester,-0.067933,jeep,excellent,6 cylinders,gas,-0.057420,clean,automatic,4wd,mid-size,sedan,white,3054,ny


In [19]:
df_test.isnull().any()

region          False
year            False
manufacturer    False
condition       False
cylinders       False
fuel            False
odometer        False
title_status    False
transmission    False
drive           False
size            False
type            False
paint_color     False
price           False
state           False
dtype: bool

In [19]:
df_test.isna().any()

region          False
year            False
manufacturer    False
condition       False
cylinders       False
fuel            False
odometer        False
title_status    False
transmission    False
drive           False
size            False
type            False
paint_color     False
price           False
state           False
dtype: bool

In [20]:
def categoryPos(df: pd.DataFrame):
    return [ df.columns.get_loc(col) for col in list(df.select_dtypes('object').columns) ]

In [21]:
def do_kproto(df: pd.DataFrame) -> pd.DataFrame:
    ctgpos = categoryPos(df)
    
    list_n_cluster = []
    list_cost = []
    for _n in tqdm.tqdm(range(1,16,1)):
        try:
            kprototype = KPrototypes(n_jobs = -1, n_clusters = _n, init = 'Huang', random_state=random_state)
            kprototype.fit_predict(df, categorical = ctgpos)
            list_n_cluster.append(_n)
            list_cost.append(kprototype.cost_)
        except Exception as error:
            print(error)
            break

    return pd.DataFrame(list(zip(list_n_cluster, list_cost)), columns=['n_cluster', 'cost'])


In [22]:
def draw_cluster_vs_cost(df: pd.DataFrame):
    fig = px.line(df, x='n_cluster', y='cost', title='KPrototypes: n_cluster vs cost')
    fig.show()

In [23]:
train_result = do_kproto(df_train)
train_result
#draw_cluster_vs_cost(train_result)

100%|███████████████████████████████████████████████████████████████████████████████████████| 15/15 [4:29:56<00:00, 1079.77s/it]


,n_cluster,cost
0,1,143344.000000
1,2,124493.612208
2,3,110723.284166
3,4,104424.478730
4,5,100586.136971
5,6,97251.652353
6,7,95298.326188
7,8,93501.272120
8,9,91913.806790
9,10,90734.440897


In [20]:
import pandas as pd
from datetime import datetime
save_datetime=datetime.now().strftime("%Y-%m-%dT%H.%M.%S.%fZ")
type(save_datetime)

str

In [45]:
type(train_result)
%pwd

'/Users/3ign0n/workspace/data-science/SIGNATE/SIGNATE_Career_Up_Challenge_2023/scuc23/notebooks'

In [44]:
def save_kprototypes_result(df: pd.DataFrame, save_datetime: str):
    output_dir_base="../data/07_model_output/kprototypes"
    output_dir=os.path.join(output_dir_base, save_datetime)
    os.makedirs(output_dir, exist_ok=True)

    df.to_csv(os.path.join(output_dir, 'kprototypes_train.csv'))


%pwd
save_kprototypes_result(df=train_result, save_datetime=save_datetime)

In [ ]:
train_result=pd.read_csv("../data/07_model_output/kprototypes/2023-08-16T21.21.37.731653Z/kprototypes_train.csv")
import plotly.express as px
fig = px.line(train_result, x='n_cluster', y='cost', title='KPrototypes: n_cluster vs cost')
fig.show()